# Project Data Science: Bitcoin Price Prediction

## 1. Introduction
In this day AI have a significant impact almost in our life and work place, and for this field we will leveraging ai in sequential data development. In this Data Science Project we will do some research about BTCUSD consolidation price with Deep Learning Neural Network LSTM method using PyTorch library
### 1. 1 Project Objective
The main objective is to predict BTC price until the next months with LSTM (Long Short Term Memory) method
### 1. 2 Dataset Description
`btcusd_1-min_data` is the name of the dataset, which was taken from the Kaggle Dataset source: https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data and has seven columns: `Timestamps, Open, Close, Min, Max, Datetime`. The data is a compilation of the price of Bitcoin from 2012 until early 2025, which is a changes every minute.